#### Recurrent Neural Networks (Recap)

- Why the need for RNNs?
    - Feedforward neural networks are not designed to handle sequential/temporal data. Not all problems can be converted into one with fixed- length inputs and outputs!
    - Feedforward neural networks (and Convolutional neural networks), can only take in a fixed sized vector as input and produce a fixed sized vector as output, using a fixed size of computational steps (layers). Recurrent Nets allows us to operate over sequences of vectors.

![RNN](https://cdn.jsdelivr.net/gh/AuthurWhywait/images/20211202145041.png)

* The networks preidctions are not only a function of the input at a particular time step, but also the past memory of cell states denoted by the hidden state (h). --> output is now a function of both the current input and the past memory at a previous time step. 

$$ h_t = f(W(x_t, h_{t-1})) $$

1. $h_t$ is a cell state;
2. $f_W$ is a function with weights $W$;
3. $x_t$ is the input;
4. $h_{t−1}$ is the old state.

NOTE: the same function and set of parameters are used at every time step in each iteration.

![RNN](https://cdn.jsdelivr.net/gh/AuthurWhywait/images/20211202151931.png)

### Goal:

- At each step the RNN needs to combine the information from the current input with the knowledge it has accumulated so far.
- Then based on a transformation of the current input and the previous hidden state, it produces an output and a new hidden state.
- The hidden state represents the RNN's memory of the sequence it has seen so far.
- This hidden state will influence the next time step, allowing the network to capture sequential information.

![title](http://karpathy.github.io/assets/rnn/diags.jpeg)

(1) Traditional Feed Forward Neural Network    
(2) Sequence Output (Image Captioning)    
(3) Sequence Input (Sentiment Analysis)    
(4) Sequence Input and Sequence Output (Machine Translation)    
(5) Synced Sequence Input and Output (Video Classification)    

Note: RNNs utilize the same set of weights at each time step, thus the number of parameters do not increase with the size of the input. This is a significant advantage when dealing with long  and variable length sequences.

![RNN](https://cdn.jsdelivr.net/gh/AuthurWhywait/images/20211202152611.png)

### Using an RNN for a Character-Level Language Model

#### What goes into the model:
- X: A sequence of characters (eg. a sentence, a document, a word)
- Example: "Hi, Hell"

#### What will the model do:
- Learn to predict the next character in the sequence. That is, we get the model to give the probability distribution of the next character in the sequence given the sequence of previous characters.
- Example: Given the sequence "Hi, Hell", the model should predict the next character as "o" with high probability.
- The model continues to predict the next character given the sequence of previous characters and the predicted characters.

Another Example:

Suppose we only had a vocabulary of four possible letters “helo”, and wanted to train an RNN on the training sequence “hello”. This training sequence is in fact a source of 4 separate training examples: 
1. The probability of “e” should be likely given the context of “h”, 
2. “l” should be likely in the context of “he”, 
3. “l” should also be likely given the context of “hel”, and finally 
4. “o” should be likely given the context of “hell”.

Image bellow: RNN is passed the sequence "hell" and it should predict the next character as "o" with high probability. (Which it does, highest score is at index 3 for "o")

![title](http://karpathy.github.io/assets/rnn/charseq.jpeg)

### How is training done?

- We train the model to minimize the cross-entropy loss between the true distribution of the next character and the predicted distribution.
- Example: When the model gets the first character 'h' (at the first time step), it should predict the next character 'e' with high probability. Thereby, we tweak the model's parameters such that it gets better at predicting the next character ('e') given the first character ('h'). And so on for the rest of the sequence.

### Note:

- Notice also that the first time the character “l” is input, the target is “l”, but the second time the target is “o”. The RNN therefore cannot rely on the input alone and must use its recurrent connection to keep track of the context to achieve this task.

### How is testing/eval done?

- At test time, we feed a character into the RNN and get a distribution over what characters are likely to come next. We sample from this distribution, and feed it right back in to get the next letter. Repeat this process and you’re sampling text!

### Problems with RNNs

![title](https://cdn.jsdelivr.net/gh/AuthurWhywait/images/20211202155526.png)

The gradient flows backwards through all the time steps (from the loss at the end to the first time step). This is called backpropagation through time (BPTT). This means the gradient computation involves recurrent multiplication of $W$. This can lead to the following problems:

- Vanishing Gradients: If the recurrent weight matrix has small singular values, the gradients will vanish as we backpropagate to earlier time steps. This means that the model is not able to capture long range dependencies in the data.

- Exploding Gradients: If the recurrent weight matrix has large singular values, the gradients will explode as we backpropagate to earlier time steps. This means that the model will not be able to learn effectively. Could be solved by clipping the gradients. (i.e. if gradient is larger than a threshold, clip it to the threshold)

Why are vanishing gradients a problem? 

- When you keep multiplying a small number by another small number, the number is going to keep shrinking and shrinking and eventually going to vanish.
- So gradients near the beginning of the sequence are going to be very small and the model is not going to learn effectively, and loosing out on long range dependencies.

### Long Short Term Memory (LSTM) Networks
- aims to solve the vanishing gradient problem

- 3 types of gates:
    - Forget gate: Decides what information to throw away from the cell state.
    - Input gate: Decides what new information to store in the cell state.
    - Output gate: Decides what to output based on input and the memory of the cell.

In standard RNN, the repeating modules contain a simple computation node:

![RNN](https://cdn.jsdelivr.net/gh/AuthurWhywait/images/20211202164941.png)

LSTM modules contain computational blocks that control information flow.

![RNN](https://cdn.jsdelivr.net/gh/AuthurWhywait/images/20211202165435.png)

Information is added or removed through structures called gates. Gates optionally let information through, for example via a sigmoid neural net layer and pointwise multiplication.

How do LSTMs work?

1. Forget. LSTMs forget irrelevant parts of the previous state.
2. Store. LSTMs store relevant new information into the cell state.
3. Update. LSTMs selectively update cell state values.
4. Output. The output gate controls what information is sent to the next time step.

![RNN](https://cdn.jsdelivr.net/gh/AuthurWhywait/images/20211202170304.png)

### 1. Forget gate: Decides what information to throw away from the cell state.

$$ f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f) $$

* This decision is made by a sigmoid layer called the “forget gate layer.” The forget gate takes the current input $x_t$ and the previous hidden state $h_{t-1}$, and applies a sigmoid function to determine which parts of the cell state to retain and which parts to forget. 

* A value of 0 means "completely forget" that element of the cell state.

* A value of 1 means "completely retain" that element.

* Values between 0 and 1 represent partial retention, meaning the element is scaled down.

* How its used:

    * The cell vector contains various values that represent the memory from the previous time step.
    * The forget gate generates a vector of values (each between 0 and 1) that is the same size as the cell state.
    * Each element of the cell state is multiplied by the corresponding value from the forget gate:
        * If the forget gate value is close to 1, that part of the cell state remains mostly unchanged.
        * If the forget gate value is close to 0, that part of the cell state is nearly "forgotten."

    * In Essence, helping the LSTM control how much past information is carried forward or discarded.

![RNN](https://cdn.jsdelivr.net/gh/AuthurWhywait/images/20211202170357.png)

### 2. Input gate: Decides what new information to store in the cell state.

- Sigmoid Activation for Filtering New Information
    - Sigmoid activation function on the input and the previous hidden state (concatenated). This acts as a filter, deciding how much of the new information to add to the cell state.    
    - Used to control the flow of new information into the cell state. Deciding at each step whether it should update the memory (cell state) with new content or stick to what it already knows.

- Candidate Cell State (Tanh Activation)
    - To generate what new information should be considered
    - The Tanh activation function creates a candidate cell state, which represents the potential new information to be added to the cell state.

- Multiplying Sigmoid and Tanh Outputs
    - The LSTM needs to control how much of the candidate cell state to add to the current cell state.
    - Sigmoid output (values between 0 and 1) gets multiplied element-wise with the candidate cell state. Thus scales the new information, allowing the LSTM to decie how much to add.
    - If sigmoid output is closer `0`, the LSTM will add less of the candidate cell state, and if closer to `1`, it will add more.
    - Ensures that only the relevant parts of the new information is introduced into long-term memory.


![RNN](https://cdn.jsdelivr.net/gh/AuthurWhywait/images/20211202170441.png)

### 3. Update

* Combine the old cell state with the new candidate values to update the cell state.

* Element-Wise Update of the Cell State
    - The LSTM's cell state is like a memory bank. To keep useful long-term info while adding new relevant updates, it must selectively combine new information with the existing memory. (Balance between retaining old information and adding new information)
    - The existting cell state is element-eise updated:
        - Some parts of the current cell state are kept (based on the forget gate’s decision)
        - New information is added (from the scaled candidate state).

![RNN](https://cdn.jsdelivr.net/gh/AuthurWhywait/images/20211202170734.png)

#### 4. Output gate: Decides what to output based on input and the memory of the cell.

- The output gate controls how much information from the cell state should be transferred to the hidden state (which is the actual output at each time step)
- The LSTM needs to decide which parts of the updated cell state are relevant to pass along for further processing, and which parts can be kept internal to the cell state.
- This step ensures that the output is a filtered version of the cell state, containing only the relevant information.

- Sigmoid Activation
    - Uses the current input and the previous hidden state to decide how much of the cell state should be used to generate the hidden state.

- Tanh Activation on the Cell State
    - The cell state (which holds the memory of the LSTM) is passed through a Tanh activation function to compress the values to be between -1 and 1.

- Elemnt-wise Multiplication
    - The LSTM needs to control how much of the scaled cell state (from the tanh function) should be included in the final hidden state at this time step.
    - controls how much of each part of the cell state becomes part of the hidden state.
    


![title](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*goJVQs-p9kgLODFNyhl9zA.gif)

Recommended Read: 

https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(RNNModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # RNN layers
        self.rnn = nn.RNN(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.rnn(x)

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :] # Get the last time step output

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        return out


class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(LSTMModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # LSTM layers
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :] # Get the last time step output

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out
        

In [5]:
import string
import nltk
from collections import Counter
import numpy as np

In [6]:
# read document
with open ('data/tinyshakespeare.txt', 'r') as f:
    doc = f.read()

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SequentialModule(nn.Module):
    # initialize module
    def __init__(self, n_vocab, seq_size=32, embedding_size=64, lstm_size=64):
        super(SequentialModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)
        
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state
    
    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),torch.zeros(1, batch_size, self.lstm_size))

In [8]:
batch_size = 16
seq_size = 32
embedding_size = 64
lstm_size = 64
gradients_norm = 5
# set device parameter
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# get list of words from document
def doc2words(doc):
    lines = doc.split('\n')
    lines = [line.strip(r'\"') for line in lines]
    words = ' '.join(lines).split()
    return words

def removepunct(words):
    punct = set(string.punctuation)
    words = [''.join([char for char in list(word) if char not in punct]) for word in words]
    return words

# get vocab from word list
def getvocab(words):
    wordfreq = Counter(words)
    sorted_wordfreq = sorted(wordfreq, key=wordfreq.get)
    return sorted_wordfreq

# get dictionary of int to words and word to int
def vocab_map(vocab):
    int_to_vocab = {k:w for k,w in enumerate(vocab)}
    vocab_to_int = {w:k for k,w in int_to_vocab.items()}
    return int_to_vocab, vocab_to_int

words = removepunct(doc2words(doc))
vocab = getvocab(words)
int_to_vocab, vocab_to_int = vocab_map(vocab)

In [10]:
def get_batches(words, vocab_to_int, batch_size, seq_size):
    # generate a Xs and Ys of shape (batchsize * num_batches) * seq_size
    word_ints = [vocab_to_int[word] for word in words]
    num_batches = int(len(word_ints) / (batch_size * seq_size))
    Xs = word_ints[:num_batches*batch_size*seq_size]
    Ys = np.zeros_like(Xs)
    Ys[:-1] = Xs[1:]
    Ys[-1] = Xs[0]
    Xs = np.reshape(Xs, (num_batches*batch_size, seq_size))
    Ys= np.reshape(Ys, (num_batches*batch_size, seq_size))
    
    # iterate over rows of Xs and Ys to generate batches
    for i in range(0, num_batches*batch_size, batch_size):
        yield Xs[i:i+batch_size, :], Ys[i:i+batch_size, :]

In [11]:
def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer
    
def generate_text(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    
    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))

In [14]:
def train_rnn(words, vocab_to_int, int_to_vocab, n_vocab):
    
    # RNN instance
    net = SequentialModule(n_vocab, seq_size, embedding_size, lstm_size)
    net = net.to(device)
    criterion, optimizer = get_loss_and_train_op(net, 0.01)

    iteration = 0
    
    for e in range(10):
        batches = get_batches(words, vocab_to_int, batch_size, seq_size)
        state_h, state_c = net.zero_state(batch_size)

        # Transfer data to GPU
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        for x, y in batches:
            iteration += 1

            # Tell it we are in training mode
            net.train()

            # Reset all gradients
            optimizer.zero_grad()

            # Transfer data to GPU
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss_value = loss.item()

            # Perform back-propagation
            loss.backward(retain_graph=True)

            _ = torch.nn.utils.clip_grad_norm_(net.parameters(), gradients_norm)
            
            # Update the network's parameters
            optimizer.step()

            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, 10),'Iteration: {}'.format(iteration),'Loss: {}'.format(loss_value))

            # if iteration % 1000 == 0:
                # predict(device, net, flags.initial_words, n_vocab,vocab_to_int, int_to_vocab, top_k=5)
                # torch.save(net.state_dict(),'checkpoint_pt/model-{}.pth'.format(iteration))
                
    return net

In [15]:
rnn_net = train_rnn(words, vocab_to_int, int_to_vocab, len(vocab))

Epoch: 0/10 Iteration: 100 Loss: 7.231979846954346
Epoch: 0/10 Iteration: 200 Loss: 6.964202880859375
Epoch: 0/10 Iteration: 300 Loss: 6.74375057220459
Epoch: 1/10 Iteration: 400 Loss: 6.750913619995117
Epoch: 1/10 Iteration: 500 Loss: 5.9644694328308105
Epoch: 1/10 Iteration: 600 Loss: 6.4356184005737305
Epoch: 1/10 Iteration: 700 Loss: 6.3372111320495605
Epoch: 2/10 Iteration: 800 Loss: 6.194395542144775
Epoch: 2/10 Iteration: 900 Loss: 5.7319560050964355
Epoch: 2/10 Iteration: 1000 Loss: 5.853903770446777
Epoch: 2/10 Iteration: 1100 Loss: 5.552889823913574
Epoch: 3/10 Iteration: 1200 Loss: 5.788702011108398
Epoch: 3/10 Iteration: 1300 Loss: 5.7780680656433105
Epoch: 3/10 Iteration: 1400 Loss: 5.656917095184326
Epoch: 3/10 Iteration: 1500 Loss: 5.560693740844727
Epoch: 4/10 Iteration: 1600 Loss: 5.155842304229736
Epoch: 4/10 Iteration: 1700 Loss: 5.145718097686768
Epoch: 4/10 Iteration: 1800 Loss: 5.2600860595703125
Epoch: 4/10 Iteration: 1900 Loss: 5.132971286773682
Epoch: 5/10 Iter

In [ ]:
generate_text(device, rnn_net, ['Today', 'marks'], len(vocab), vocab_to_int, int_to_vocab)

Today marks you sir what you do thee no of my masters use it was the duke is that I have no screen between thee to my heart LUCENTIO Tranio and my father Pedant I pray Bianca And I pray between your hands the devils musician your name and beat him from the house of the street o it on thee and so I have seen a farmers ground was a very apoplexy pound or the rest that eer that you are choleric Take thy life and therefore shall not have taen him Pedant Sir let the time KATHARINA No my son Lucentio


How to Generate?

- Greedy decoding: Always pick the word with the highest probability 

- Sampling: Sample a word according to the given distribution

- Beam search:

### Beam Search:

Beam search is a strategy used in algorithms that need to make a series of choices from potentially very large sets of options, such as generating sentences word by word in natural language processing.

At every step, predicting a different word, could lead the text generated in a completely different path, and most likely there are far too many paths for the model to explore all of them. 

Beam search helps by keeping track of a limited number of promosing paths (incomplete sentences), at each step, and extends those paths with new words (until the end of the sequence or maximum length is reached).


#### Greedy Search

![title](https://huggingface.co/blog/assets/02_how-to-generate/greedy_search.png)

Starting from the word "The", the algorithm greedily chooses the next word of highest probability "nice", and so on, so that the final generated word sequence is ("The", "nice", "woman") having an overall probability of 0.5 × 0.4 = 0.2.

![title](https://huggingface.co/blog/assets/02_how-to-generate/beam_search.png)

At time step 1, besides the most likely hypothesis ("The", "nice"), beam search also keeps track of the second most likely one ("The", "dog"). At time step 2, beam search finds that the word sequence ("The", "dog", "has") has with 0.36 a higher probability than ("The", "nice", "woman"), which has 0.2.

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

def generate_text_beam_search(model, tokenizer, prompt_text, beam_width=5, max_length=100):
    def beam_search_step(input_ids, cum_log_probs):
        outputs = model(input_ids=input_ids)
        logits = outputs.logits[:, -1, :]
        probs = torch.softmax(logits, dim=-1)
        log_probs, next_tokens = torch.topk(probs, beam_width, dim=-1)
        return log_probs, next_tokens

    input_ids = tokenizer.encode(prompt_text, return_tensors='pt')
    beam_candidates = [(input_ids, 0.0, 0)]  # (input_ids, cumulative_log_prob, length)
    completed_sequences = []

    with torch.no_grad():
        for step in range(max_length):
            all_candidates = []
            for input_ids, cum_log_prob, length in beam_candidates:
                log_probs, next_tokens = beam_search_step(input_ids, cum_log_prob)
                for i in range(beam_width):
                    new_input_ids = torch.cat([input_ids, next_tokens[:, i].unsqueeze(0)], dim=-1)
                    new_cum_log_prob = cum_log_prob + log_probs[:, i].item()
                    new_length = length + 1

                    if next_tokens[:, i].item() == tokenizer.eos_token_id:
                        perplexity = torch.exp(torch.tensor(-new_cum_log_prob) / new_length)
                        completed_sequences.append((new_input_ids, perplexity))
                    else:
                        all_candidates.append((new_input_ids, new_cum_log_prob, new_length))
            beam_candidates = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width]
            if len(completed_sequences) >= 1:  # Break if at least one sequence is completed
                break
    
    # beam_candidate_sentences = [tokenizer.decode(candidate[0][0], skip_special_tokens=True) for candidate in beam_candidates]
    # Choose the sequence with the lowest perplexity among completed sequences
    if completed_sequences:
        best_sequence = min(completed_sequences, key=lambda x: x[1])
        return tokenizer.decode(best_sequence[0][0], skip_special_tokens=True), best_sequence[1].item()
    else:
        # If no sequences were completed, return the best of the current candidates
        best_sequence = min(beam_candidates, key=lambda x: x[1])
        return tokenizer.decode(best_sequence[0][0], skip_special_tokens=True)

# Initialization
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', cache_dir='/projectnb/ds598/projects/xthomas/misc')
model = GPT2LMHeadModel.from_pretrained('gpt2', cache_dir='/projectnb/ds598/projects/xthomas/misc')


# Generating text with different beam widths
prompt = "As the sun set over the horizon,"
beam_widths = [3, 5, 10, 20]
max_length = 50

for width in beam_widths:
    print(f"\nBeam Width: {width}")
    text, perplexity = generate_text_beam_search(model, tokenizer, prompt, beam_width=width, max_length=max_length)
    print(f"Generated Text: {text}")



Beam Width: 3
Generated Text: As the sun set over the horizon, the sky was filled with clouds.

"What's going on here?"

"I don't know."

"What's going on here?"

"I don't know."

"What's going on?"

Beam Width: 5
Generated Text: As the sun set over the horizon, the sky was filled with light.

"It's been a long time since I've seen anything like this."
.

Beam Width: 10
Generated Text: As the sun set over the horizon, there was a flash of light in the middle of the street.

"What's going on?" I asked.
.

Beam Width: 20
Generated Text: As the sun set over the horizon, there was nowhere to be found.

There was nowhere to hide.
